<h1>System Evaluation Results Calculation Using DataFrame</h1>
<br/>
<p>This notebook/project focuses on computing the system evaluation results for my undergraguate thesis using my skills in Python's DataFrame.</p>
<p>Specifically, it aims to:</p>
<ul>
    <li>Utilize NumPy and pandas' DataFrame for efficient data manipulation;</li>
    <li>Calculate the weighted average and standard deviation for each criterion; and</li>
    <li>Determine the overall average and standard deviation of the system evaluation. </li>
</ul>

<h2>Data Preparation</h2>
<h3>Import the thesis evaluation tally dataset.</h3>

In [3]:
# Import the libraries needed
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt

# Import the CSV file and save as DataFrame
csv = pd.read_csv('thesis-evaluation.csv')
df = pd.DataFrame(csv)

# Drop the last column
df = df.drop(columns=['Total Respondents'])

# Set indices
df_sys_eval = df.set_index(['Indicator', 'Criteria'])

# Display DataFrame
df_sys_eval.head() # Remove '.head()' to view whole table

,,Poor,Fair,Satisfactory,Very Satisfactory,Outstanding
Indicator,Criteria,,,,,
01.a,Functional Suitability,0,0,1,22,57
01.b,Functional Suitability,0,0,4,23,53
01.c,Functional Suitability,0,0,2,24,54
01.d,Functional Suitability,0,0,3,29,48
02.a,Functional Suitability,0,0,2,25,53


<h3>Prepare the given (constant) values.</h3>

In [5]:
# Total respondents (constant)
num_of_respondents = 80

# Weights/Ratings (w)
w = [1,2,3,4,5]
# 1-Poor, 2-Fair, 3-Satisfactory, 4-Very Satisfactory, 5-Outstanding

In [6]:
# Define a function to return the adjectival rating based on the Average of each row
def adj_rating(x):
    if x >= 4.21:
        return 'Outstanding'
    elif x >= 3.41:
        return 'Very Satisfactory'
    elif x >= 2.61:
        return 'Satisfactory'
    elif x >= 1.81:
        return 'Fair'
    else:
        return 'Poor'

<h2>Computation of Weighted Mean</h2>

<h3>Compute the weighted scores for each response scale.</h3>

Calculate the individual scores by multiplying the numerical values assigned to each response scale (e.g., 1-Poor, 2-Fair, ..., 5-Outstanding) by the corresponding number of responses.

In [9]:
# Compute: Summation of w(x)

# Computed weights(w) of value(x)
df_wx = df_sys_eval[['Poor', 'Fair', 'Satisfactory', 'Very Satisfactory', 'Outstanding']] * w
df_wx.head() # Remove '.head()' to view whole table

,,Poor,Fair,Satisfactory,Very Satisfactory,Outstanding
Indicator,Criteria,,,,,
01.a,Functional Suitability,0,0,3,88,285
01.b,Functional Suitability,0,0,12,92,265
01.c,Functional Suitability,0,0,6,96,270
01.d,Functional Suitability,0,0,9,116,240
02.a,Functional Suitability,0,0,6,100,265


<h3>Solve for the average scores of each indicator.</h3>

Calculate it by getting the sum of individual scores for each indicator, then divide it by the number of respondents/sample.

In [11]:
# Get the Sum of wx (per row)
df_sum_wx = df_wx.agg('sum', axis=1)

In [12]:
# Get the mean per indicator
df_mean_per_indicator = df_sum_wx.div(num_of_respondents).reset_index().rename(columns={0: 'Average'})
df_mean_per_indicator.head() # Remove '.head()' to view whole table

,Indicator,Criteria,Average
0,01.a,Functional Suitability,4.7000
1,01.b,Functional Suitability,4.6125
2,01.c,Functional Suitability,4.6500
3,01.d,Functional Suitability,4.5625
4,02.a,Functional Suitability,4.6375


<h2>Computation of Standard Deviation (STDDEV)</h2>
<h3>Prepare the DataFrame for calculation of STDDEV.</h3>

Create a copy of "df_mean_per_indicator", then insert five (5) temporary columns for easier aggregation of values needed in the formula.

In [14]:
# Create an independent copy of DataFrame
df_rate_mean_pow = df_mean_per_indicator.copy(deep=True) 

In [15]:
# Name of temporary columns that will be used for coputing STDDEV
temp_columns = ['X(1-M)^2','X(2-M)^2','X(3-M)^2','X(4-M)^2','X(5-M)^2']

df_rate_mean_pow[temp_columns] = 0 # Create new  columns with value 0

<h3>Solve for the standard deviation of each indicator.</h3>

Formula: 
STDDEV = sqrt( ∑ (Number of Votes X (Rating- Mean)^2) / N-1 )

In [42]:
'''
Formula:

'''

# Compute (Rating - Mean)^2 
for i, col in enumerate(temp_columns):
    df_rate_mean_pow[col] = df_rate_mean_pow[['Average']].sub(w[i]).pow(2)

tbl_X = df_sys_eval.copy(deep=True).reset_index(drop=True)
tbl_r_m_pow = df_rate_mean_pow[temp_columns]
df_X_r_m_pow = pd.DataFrame(tbl_X.values*tbl_r_m_pow.values)

# Get the 'Summation of (Number of Votes [X] (Rating- Mean)^2)''
df_summation = df_X_r_m_pow.agg(['sum'], axis=1)

# Create new variable for numerator and denominator values of STDDEV formula
df_std_numerator = df_summation
df_std_denominator = num_of_respondents-1

df_std_per_indicator = np.sqrt(df_std_numerator.div(df_std_denominator, axis=1))

# Drop columns for cleaner dataframe, then create new variable for easier identification
df_calc_per_indicator = df_rate_mean_pow.drop(temp_columns, axis=1)

df_calc_per_indicator['Standard Deviation'] = df_std_per_indicator
df_calc_per_indicator.head()

,Indicator,Criteria,Average,Standard Deviation
0,01.a,Functional Suitability,4.7000,0.487826
1,01.b,Functional Suitability,4.6125,0.584477
2,01.c,Functional Suitability,4.6500,0.530106
3,01.d,Functional Suitability,4.5625,0.570226
4,02.a,Functional Suitability,4.6375,0.533528


In [17]:
# Aggregate values by criteria in 'Average' and 'Standard Deviation'
df_calc_per_criteria = df_calc_per_indicator.groupby('Criteria', sort=False)[['Average', 'Standard Deviation']].mean()

In [18]:
# Calculate the overall average for 'Average' and 'Standard Deviation'
overall_avg = df_calc_per_criteria[['Average', 'Standard Deviation']].mean()

# Append it in the DataFrame df_calc_per_criteria
df_calc_per_criteria.loc['Overall'] = overall_avg

# Round to two decimal places
df_calc_per_criteria = df_calc_per_criteria.round(2)

In [19]:
# Add the Adjectival Rating per Criteria
df_calc_per_criteria['Adjectival Rating'] = df_calc_per_criteria['Average'].apply(adj_rating)

df_calc_per_criteria

,Average,Standard Deviation,Adjectival Rating
Criteria,,,
Functional Suitability,4.60,0.55,Outstanding
Performance Efficiency,4.47,0.58,Outstanding
Compatibility,4.38,0.60,Outstanding
Usability,4.60,0.56,Outstanding
Reliability,4.49,0.60,Outstanding
Security,4.50,0.62,Outstanding
Maintainability,4.52,0.63,Outstanding
Portability,4.67,0.51,Outstanding
Overall,4.53,0.58,Outstanding
